In [1]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\001\\glasses'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".avi" in file:
            files.append(os.path.join(r, file))
        
for f in files:
    print(f)
np.array(files).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\yawning.avi


(4,)

In [2]:
filename[54:-4]

NameError: name 'filename' is not defined

## MTCNN

if face not detected in 60 frames, use the last detected 

In [3]:
from mtcnn.mtcnn import MTCNN
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import numpy as np
import os
from operator import itemgetter
import time


def CreateFolder(name):
    try:
        # creating a folder named data 
        if not os.path.exists(name):
            os.makedirs(name)
        # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data')            
#==============================================================

detector = MTCNN()

for filename in files:
    # img = cv2.imread("me.jpg")
    
    #filename = '.\\Dataset\\Training_Evaluation_Dataset\\Training Dataset\\001\\glasses\\yawning.avi'

    print(filename)
    cap = cv2.VideoCapture(filename)
    #================================================================ create new folder

    DefaultPath = '.\\Dataset\\Training_Evaluation_Dataset\\Training Image\\'+filename[54:-4]+"\\"
    CreateFolder(DefaultPath+"face")
    CreateFolder(DefaultPath+"eyes")
    CreateFolder(DefaultPath+"mouth")
    #================================================================
    Face_ImageDataList = []

    currentframe = 0
    missing_frame = 0
    DropList = []

    eyes_w = 180
    eyes_h = 100
    face_w = 210
    face_h = 250
    mouth_w = 150
    mouth_h = 100
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(length)
    x,y,w,h = 0,0,0,0
    
    while(cap.isOpened()):

        start_time = time.time()

        currentframe += 1
        #if(currentframe<1700):
        #    continue
        # load the input image and convert it to grayscale
        ret, image = cap.read()
        if (not ret):
          break

        height,width = image.shape[:2]
        #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #image = cv2.resize(image,(width//2,height//2),interpolation=cv2.INTER_CUBIC)
        #print(image.shape[:2])
        faces = detector.detect_faces(image)
        #print(np.array(faces).shape[0])
        
        if(np.array(faces).shape[0] > 0):
            face = faces[0]
            x,y,w,h = face['box']

            face_image = image[y:y+h,x:x+w]

            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face/frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[face['keypoints']['left_eye'][1]-int(h*0.2):face['keypoints']['right_eye'][1]+int(h*0.2)
                                ,face['keypoints']['left_eye'][0]-int(w*0.2):face['keypoints']['right_eye'][0]+int(w*0.2)]
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes/frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[face['keypoints']['mouth_left'][1]-int(h*0.2):face['keypoints']['mouth_right'][1]+int(h*0.2)
                               ,face['keypoints']['mouth_left'][0]-int(w*0.2):face['keypoints']['mouth_right'][0]+int(w*0.2)]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth/frame" + str(currentframe) + '.jpg',mouth_image) 



            #cv2.rectangle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)


            #cv2.circle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['right_eye'][0], face['keypoints']['right_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['nose'][0], face['keypoints']['nose'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_left'][0], face['keypoints']['mouth_left'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_right'][0], face['keypoints']['mouth_right'][1]), 2, (0, 255, 0), -1)
            if(currentframe%100==0):        
                name = './data/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
        elif (missing_frame<60):
            print ('missing using last frame') 
            missing_frame = missing_frame + 1        

            face_image = image[y:y+h,x:x+w]

            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face/frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[face['keypoints']['left_eye'][1]-int(h*0.2):face['keypoints']['right_eye'][1]+int(h*0.2)
                                ,face['keypoints']['left_eye'][0]-int(w*0.2):face['keypoints']['right_eye'][0]+int(w*0.2)]
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes/frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[face['keypoints']['mouth_left'][1]-int(h*0.2):face['keypoints']['mouth_right'][1]+int(h*0.2)
                               ,face['keypoints']['mouth_left'][0]-int(w*0.2):face['keypoints']['mouth_right'][0]+int(w*0.2)]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth/frame" + str(currentframe) + '.jpg',mouth_image) 
            
            #if(currentframe%100==0):
            name = './data/frame' + str(currentframe) + '.jpg'
            print ('missing using last frame:  '+ 'Creating...' + name) 

        else:
            DropList.append(currentframe)
            print("drop")



        '''
        cv2.imshow('img',image)

        #print("FPS: ", 1.0 / (time.time() - start_time))

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        '''
    
    print("total_frame:" +str(length)+" missing frame"+ str(np.array(DropList).shape[0]))
    print(DropList)
    cap.release()
    cv2.destroyAllWindows()

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
3330
Creating..../data/frame100.jpg
Creating..../data/frame200.jpg
Creating..../data/frame300.jpg
Creating..../data/frame400.jpg
Creating..../data/frame500.jpg
Creating..../data/frame600.jpg
Creating..../data/frame700.jpg
Creating..../data/frame800.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1000.jpg
Creating..../data/frame1100.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1300.jpg
Creating..../data/frame1400.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1600.jpg
Creating..../data/frame1700.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame1900.jpg
Creating..../data/frame2000.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2200.jpg
Creating..../data/frame2300.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2500.jpg
Creating..../data/frame2600.jpg
Creating..../data/frame2700.jpg
Creating..../data/frame2800.jpg
Creating..../dat

### TODO
1. 所有file 存檔案
2. training 進CNN model
3. 存沒有東西的frame